In [17]:
#r "nuget: Microsoft.SemanticKernel, 0.11.146.1-preview"

Installed Packages Microsoft.SemanticKernel, 0.12.207.1-preview

In [18]:
#!import Skills/CustomSkill/CompanySearchSkill.cs

In [19]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;

In [20]:
using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.Orchestration;

In [21]:
IKernel kernel = Kernel.Builder.Build();

In [22]:
kernel.Config.AddAzureOpenAITextCompletionService(
    "GPT3",      
    "text-davinci-003",                  
    "Your Endpoint",    
    "Your OpenAI API Key"          
);

In [23]:
var planner = kernel.ImportSkill(new PlannerSkill(kernel));

In [24]:
planner

key value BucketOutputs Microsoft.SemanticKernel.Orchestration.SKFunction Name BucketOutputs SkillName _GLOBAL_FUNCTIONS_ Description When the output of a function is too big, parse the output into a number of buckets. IsSemantic False RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 256 StopSequences Parameters index value 0 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name input Description The output from a function that needs to be parse into buckets. DefaultValue 1 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name bucketCount Description The number of buckets. DefaultValue 2 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name bucketLabelPrefix Description The target label prefix for the resulting buckets. Result will have index appended e.g. bucketLabelPrefix='Result' => Result_1, Result_2, Result_3 DefaultValue Result CreatePlan Microsoft.SemanticKernel.Orchestration.SKFunction Name CreatePlan SkillName _GLOBAL_FUNCTIONS_ Description Create a plan using registered functions to accomplish a goal. IsSemantic False RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 256 StopSequences Parameters index value 0 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name input Description The goal to accomplish. DefaultValue 1 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name relevancyThreshold Description The relevancy threshold when filtering registered functions. DefaultValue 2 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name MaxRelevantFunctions Description Limits the number of relevant functions as result of semantic search included in the plan creation request. DefaultValue 100 3 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name excludedFunctions Description A list of functions to exclude from the plan creation request. DefaultValue 4 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name excludedSkills Description A list of skills to exclude from the plan creation request. DefaultValue 5 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name includedFunctions Description A list of functions to include in the plan creation request. DefaultValue 6 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name useConditionals Description Use conditional functions to create the plan. DefaultValue False ExecutePlan Microsoft.SemanticKernel.Orchestration.SKFunction Name ExecutePlan SkillName _GLOBAL_FUNCTIONS_ Description Execute a plan that uses registered functions to accomplish a goal. IsSemantic False RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 256 StopSequences Parameters

In [25]:

CompanySearchSkill customSkill = new CompanySearchSkill();

In [26]:
var custom_skill = kernel.ImportSkill (customSkill, "CustomSkill");

In [27]:
var skillsDirectory =  System.IO.Directory.GetCurrentDirectory() + "/Skills";

In [28]:
var write_skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory,"WriteSkill");
var email_skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory,"EmailSkill");

In [29]:
var ask = "查找广州天气，把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件";
var plan = await kernel.RunAsync(ask, planner["CreatePlan"]);

In [30]:
plan.Variables.ToPlan().PlanString

<goal>
查找广州天气，把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件
</goal>
Error: InvalidRequest: The request is not valid, HTTP status: 404

In [31]:
var executionResults = plan;

In [32]:
int step = 1;
int maxSteps = 30;
while (!executionResults.Variables.ToPlan().IsComplete && step < maxSteps)
{
    var results = await kernel.RunAsync(executionResults.Variables, planner["ExecutePlan"]);
    if (results.Variables.ToPlan().IsSuccessful)
    {
        Console.WriteLine($"Step {step} - Execution results:\n");
        Console.WriteLine(results.Variables.ToPlan().PlanString);

        if (results.Variables.ToPlan().IsComplete)
        {
            Console.WriteLine($"Step {step} - COMPLETE!");
            Console.WriteLine(results.Variables.ToPlan().Result);
            break;
        }
    }
    else
    {
        Console.WriteLine($"Step {step} - Execution failed:");
        Console.WriteLine(results.Variables.ToPlan().Result);
        break;
    }
    
    executionResults = results;
    step++;
    Console.WriteLine("");
}

Step 1 - Execution results:

<goal>
查找广州天气，把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件
</goal><plan>
</plan>
Step 1 - COMPLETE!
{"id":"534431ea6be64d65a401a9a7cf2d9bd3","goal":"\u67E5\u627E\u5E7F\u5DDE\u5929\u6C14\uFF0C\u628A\u7ED3\u679C\u7FFB\u8BD1\u6210\u4E2D\u6587\u540E\u6839\u636E\u5929\u6C14\u60C5\u51B5\u751F\u6210\u7A7F\u8863\u63D0\u793A\uFF0C\u5E76\u7ED3\u5408\u5929\u6C14\u7ED3\u679C\u548C\u7A7F\u8863\u63D0\u793A\u5199\u4E00\u5C01\u90AE\u4EF6","plan":"\u003Cgoal\u003E\n\u67E5\u627E\u5E7F\u5DDE\u5929\u6C14\uFF0C\u628A\u7ED3\u679C\u7FFB\u8BD1\u6210\u4E2D\u6587\u540E\u6839\u636E\u5929\u6C14\u60C5\u51B5\u751F\u6210\u7A7F\u8863\u63D0\u793A\uFF0C\u5E76\u7ED3\u5408\u5929\u6C14\u7ED3\u679C\u548C\u7A7F\u8863\u63D0\u793A\u5199\u4E00\u5C01\u90AE\u4EF6\n\u003C/goal\u003E\nError: InvalidRequest: The request is not valid, HTTP status: 404","arguments":"","is_complete":false,"is_successful":false,"result":""}
